### Request Structure

![Structure](https://i.imgur.com/XbubqrG.png)

### Request Path

![Paths](https://i.imgur.com/MPyc04h.png)  

- **Context Path:** The path prefix associated with the `ServletContext` that this servlet is a part of. If this context is the “default” context rooted at the base of the Web server’s URL name space, this path will be an empty string. Otherwise, if the context is not rooted at the root of the server’s name space, the path starts with a / character but does not end with a / character.
- **Servlet Path:** The path section that directly corresponds to the mapping which activated this request. This path starts with a ’/’ character except in the case where the request is matched with the ‘/\*’ or ““ pattern, in which case it is an empty string. 
- **PathInfo:** The part of the request path that is not part of the Context Path or the Servlet Path. It is either null if there is no extra path, or is a string with a leading ‘/’.  

The following methods exist in the HttpServletRequest interface to access this information:
- `getContextPath`
- `getServletPath`
- `getPathInfo`

### Request Parameter
In servlet spec request parameter can refer to both url parameters and post form body.  
**Form Data:** form data can either be
- multipart/form-data: use this if form contains binary data (like file uploads) or big forms.
- application/x-www-form-urlencoded: small text form data. This is the one that can be obtained using the below code example. The body of the HTTP message sent to the server is essentially one giant query string -- name/value pairs are separated by the ampersand

In [ ]:
// Request parameter
writer.write("<br><b>Request Parameters</b><br>");
Enumeration<String> reqParamNames = request.getParameterNames();
while(reqParamNames.hasMoreElements()) {
    String param = reqParamNames.nextElement();
    writer.write( param + " : " + request.getParameter(param) + "<br>");
}

If we have a url like http://localhost:8080/WebProject2.5/requestinfo?language=en&location=in, request parameter would be (language, en) and (location,in). We can differentiate a bit between these two by parsing query string.

There is no direct way to obtain path parameters, one would need to parse the request URI obtained using `getRequestURI` method or the `getPathInfo` method.

In [ ]:
writer.write(request.getQueryString() + "<br>");

To read request body:

In [ ]:
// Read request body
BufferedReader reader = new BufferedReader(req.getReader());
StringBuilder builder = new StringBuilder();

String line = "";
while ((line = reader.readLine()) != null) {
    builder.append(line);
}

### Request Attribute
Request attribute is essentially a name value pair used by a servlet to communicate with another servlet.

In [ ]:
request.getAttribute(attributeName);
request.setAttribute(attributeName, attributeValue);

### Headers
To get request headers defined in the request,

In [ ]:
StringBuilder respStr = new StringBuilder();
Enumeration<String> reqHeaders = request.getHeaderNames();
while (reqHeaders.hasMoreElements()) {
    String header = reqHeaders.nextElement();
    respStr.append("<b>" + header + "</b>" + ":" + request.getHeader(header) + "<br>");
}

### Cookies
To get cookies associated with a request:

In [ ]:
Cookie[] cookies = request.getCookies();
for (Cookie cookie : cookies) {
    respStr.append(
            "<tr><td>" + cookie.getName() + "</td><td>" + cookie.getValue() + "</td><td>" + cookie.getPath()
                    + "</td><td>" + cookie.getDomain() + "</td><td>" + cookie.getMaxAge() + "</td></tr>");
}

**Cookie Primer:** a server can request client to save a cookie by using the following header:  
Cookie with expiration time (cookie without expires or max-age is a session cookie)
```
Set-Cookie: id=a3fWa; Expires=Thu, 31 Oct 2021 07:28:00 GMT;
```
Cookie with max age (in seconds)
```
Set-Cookie: id=re34df; Max-Age=300
```
Secure cookie can only be sent over HTTPS connection. Cookie with the HttpOnly attribute is inaccessible to the JavaScript `Document.cookie` API
```
Set-Cookie: id=a3fWa; Expires=Thu, 21 Oct 2021 07:28:00 GMT; Secure; HttpOnly
```
Domain attribute specifies which hosts can receive a cookie. If the server does not specify a Domain, the browser defaults the domain to the same host that set the cookie, excluding subdomains. Path attribute indicates a URL path that must exist in the requested URL in order to send the Cookie header.

### Locale
Client may indicate what language they would prefer the response be given in using the `Accept-Language`  header along with other mechanisms described in the HTTP/1.1 specification. `ServletRequest` interface provides following to determine the preferred locale of the sender:
- getLocale
- getLocales

### Request Dispatcher
Request dispatcher helps to transfer request from one servlet to another. The request then either comes back (include), or the new servlet sends the response (forward).  

![include](https://i.imgur.com/T9RrGmN.png)

![forward](https://i.imgur.com/HAGxiSm.png)

The key difference between the two is that the forward will CLOSE the output stream after it has been invoked, whereas the include leaves the output stream OPEN.

In [ ]:
RequestDispatcher rd = request.getRequestDispatcher("Processor.jsp");
rd.forward(request,response);
out.println("back to servlet"); // this won't be displayed

### Request Object Lifecycle
Each request object is valid only within the scope of a servlet’s `service` method, or within the scope of a filter’s `doFilter` method, unless the asynchronous processing is enabled for the component and the `startAsync` method is invoked on the request object. In the case where asynchronous processing occurs, the request object remains valid until `complete` is invoked on the `AsyncContext`. 

Containers commonly recycle request objects in order to avoid the performance overhead of request object creation. The developer must be aware that maintaining references to request objects for which `startAsync` has not been called outside the scope described above is not
recommended as it may have indeterminate results.